In [ ]:
!pip install Sastrawi tensorflow

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ArmFriiz/Dicoding-Submission-FDL/refs/heads/main/Analisis%20Sentimen/dataset_ulasan_playstore.csv')

In [ ]:
df.head(5)

,content,score,label
0,CEPAT AMAN TERPERCAYA,5,Positif
1,top markotop,5,Positif
2,belanja online nomor 1 satu di Indonesia.,5,Positif
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,1,Negatif
4,jadi langganan belanja di shopee..,5,Positif


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73478 entries, 0 to 73477
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  73478 non-null  object
 1   score    73478 non-null  int64 
 2   label    73478 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [ ]:
df.describe(include='all')

,content,score,label
count,73478,73478.000000,73478
unique,72705,NaN,3
top,bagus,NaN,Positif
freq,5,NaN,51173
mean,NaN,3.862802,NaN
std,NaN,1.668418,NaN
min,NaN,1.000000,NaN
25%,NaN,2.000000,NaN
50%,NaN,5.000000,NaN
75%,NaN,5.000000,NaN


In [ ]:
# df.drop(columns=['score'], inplace=True)

In [ ]:
df.columns

Index(['content', 'score', 'label'], dtype='object')

In [ ]:
def cleaning_text(text):
    text = text.lower() # Ubah teks menjadi huruf kecil
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Hapus mention
    text = re.sub(r'http\S+', '', text) # Hapus URL
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
    text = re.sub(r"(.)\1+", r"\1\1", text) # Hapus kata berulang

    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    text = ' '.join(text.split()) # Bersihkan spasi berlebih di tengah dan di ujung
    return text

In [ ]:
def stemmingText(text, stemmer): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Memecah teks menjadi daftar kata
    words = text

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

In [ ]:
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

In [ ]:
def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    # listStopwords1 = set(stopwords.words('english'))
    # listStopwords.update(listStopwords1)
    # listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
df['clean_content'] = df['content'].apply(cleaning_text)
df['tokenized_content'] = df['clean_content'].apply(tokenizingText)
df['filtered_content'] = df['tokenized_content'].apply(filteringText)
# df['stemmed_content'] = df['filtered_content'].apply(lambda x: stemmingText(x, stemmer))

In [ ]:
df

,content,score,label,clean_content,tokenized_content,filtered_content
0,CEPAT AMAN TERPERCAYA,5,Positif,cepat aman terpercaya,"[cepat, aman, terpercaya]","[cepat, aman, terpercaya]"
1,top markotop,5,Positif,top markotop,"[top, markotop]","[top, markotop]"
2,belanja online nomor 1 satu di Indonesia.,5,Positif,belanja online nomor satu di indonesia,"[belanja, online, nomor, satu, di, indonesia]","[belanja, online, nomor, indonesia]"
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,1,Negatif,ganggu doang lu org lagi main juga,"[ganggu, doang, lu, org, lagi, main, juga]","[ganggu, doang, lu, org, main]"
4,jadi langganan belanja di shopee..,5,Positif,jadi langganan belanja di shopee,"[jadi, langganan, belanja, di, shopee]","[langganan, belanja, shopee]"
...,...,...,...,...,...,...
73473,murah pokoknu beli di shopee,5,Positif,murah pokoknu beli di shopee,"[murah, pokoknu, beli, di, shopee]","[murah, pokoknu, beli, shopee]"
73474,"selalu puas belanja di shopee, mudah di aplika...",5,Positif,selalu puas belanja di shopee mudah di aplikas...,"[selalu, puas, belanja, di, shopee, mudah, di,...","[puas, belanja, shopee, mudah, aplikasikan, to..."
73475,"berbelanja murah, pengiriman cepat dan pelayan...",4,Positif,berbelanja murah pengiriman cepat dan pelayana...,"[berbelanja, murah, pengiriman, cepat, dan, pe...","[berbelanja, murah, pengiriman, cepat, pelayan..."
73476,"sejak shopee indonesia ada, yg namanya outfit,...",5,Positif,sejak shopee indonesia ada yg namanya outfit s...,"[sejak, shopee, indonesia, ada, yg, namanya, o...","[shopee, indonesia, yg, namanya, outfit, sepat..."


In [ ]:
top_20 = df['tokenized_content'].explode().value_counts().head(20)
top_20

,count
tokenized_content,
shope,30841
di,26419
dan,16975
saya,14518
sangat,12695
nya,12620
belanja,11954
barang,10130
yg,9695
